In [ ]:
# 구글 드라이브 연동 + GPU연결
from google.colab import drive
drive.mount("/content/drive")

# 라이브러리
import os
import tensorflow as tf
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

# 1. VGG16 모델 생성
base_model = tf.keras.applications.vgg16.VGG16(
    include_top = False,
    weights = 'imagenet',
    input_shape = (224, 224, 3)
)
base_model.summary()                                                            # 모델 구조 확인

for layer in base_model.layers:                                                 # VGG16 모델 계층 동결
    layer.trainable = False

# 2. 모델 미세 조정
labels = 2                                                                      # 출력 레이블 개수 설정
inputs = base_model.input
x = base_model.output                                                           # 전체 모델의 중간 출력/ 분류기 층의 입력
x = tf.keras.layers.GlobalAveragePooling2D()(x)                                 # feature 개수 축소
x = tf.keras.layers.BatchNormalization()(x)                                     # 층의 입력 정규화
x = tf.keras.layers.Dropout(rate = 0.5)(x)                                      # 뉴런 일부만 연결
x = tf.keras.layers.Dense(256, activation = 'relu',
                          kernel_initializer = 'he_normal',                     # 가중치 값 초기 설정
                          bias_initializer = 'zero')(x)                         # bias 초기 설정
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(rate = 0.5)(x)

prediction = tf.keras.layers.Dense(units = labels, activation = 'softmax')(x)
finetuned_model = tf.keras.Model(inputs = inputs, outputs = prediction)

finetuned_model.summary()

# 3. 학습 데이터 증식
train_path = '/content/drive/MyDrive/Images/cats_and_dogs/train'                 # 이미지 폴더 경로
val_path = '/content/drive/MyDrive/Images/cats_and_dogs/val'

HEIGHT = 224                                                                    # 이미지 크기 설정
WIDTH = 224

train_generated = tf.keras.preprocessing.image.ImageDataGenerator(              # 학습용 데이터 증식
    rescale = 1/255,                                                            # normalization
    rotation_range = 30,                                                        # 회전
    width_shift_range = 0.2,                                                    # 수평 이동
    height_shift_range = 0.2,                                                   # 수직 이동
    shear_range = 0.2,                                                          # 비틀림 강도
    zoom_range = 0.2,                                                           # 크기 변화
    horizontal_flip = True,                                                     # 좌우 반전
    vertical_flip = True,                                                       # 상하 반전
    fill_mode = 'nearest')

val_generated = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1/255)

train_generator = train_generated.flow_from_directory(                          # 학습용 입력 데이터 생성
    train_path,
    batch_size = 60,
    target_size = (HEIGHT, WIDTH),
    shuffle = True,                                                             # 임의로 이미지 추출
    class_mode = 'categorical')

val_generator = val_generated.flow_from_directory(                              # 검증용 입력 데이터 생성
    val_path,
    target_size = (HEIGHT, WIDTH),
    batch_size = 60,
    shuffle = True,
    class_mode = 'categorical')

# 4. 학습 진행
checkpoint_dir = '/content/drive/MyDrive/Images'                                # 결과 저장 경로 설정
file_name = 'fineruned_weight.hdf5'

early_stop = tf.keras.callbacks.EarlyStopping(                                  # 조기 종료 설정
    monitor = 'val_loss',
    patience = 5,
    mode = 'min',
    verbose = 1)                                                                # 학습과정 출력

checkpoint = tf.keras.callbacks.ModelCheckpoint(                                # 최적의 학습 결과 저장 설정
    filepath = checkpoint_dir + '/' + file_name,
    monitor = 'val_loss',
    mode = 'min',
    verbose = 1,
    save_best_only = True)

optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0001)                    # 모델 컴파일
model.compile(loss = 'categorical_crossentropy',
              optimizer = optimizer,
              metrics = ['accuracy'])

history = model.fit(train_generator,                                            # 학습 진행
                    validation_data = val_generator,
                    epochs = 100000,
                    callbacks = [early_stop, checkpoint])

# 5. 학습 결과 시각화
acc = history.history['accuracy']                                               # y축 지정
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = np.arange(1, len(acc) + 1)                                             # X축 지정

plt.plot(epochs, acc, 'b', label = 'Training acc')                              # 정확도 그래프
plt.plot(epochs, val_acc, 'g', label = "Validation acc")
plt.title("Accuracy")
plt.legend()
plt.show()

plt.plot(epochs, loss, 'b', label = 'Training loss')                            # 손실 그래프
plt.plot(epochs, val_loss, 'g', label = "Validation loss")
plt.title("Loss")
plt.legend()
plt.show()

# 6. 모델 성능 확인
model_path = '/content/drive/MyDrive/Images/fineruned_weight.hdf5'                   # 모델 불러오기
tuned_model = tf.keras.models.load_model(model_path)

# 7. 테스트용 이미지 준비
test_dir = '/content/drive/MyDrive/ImagesProcessing/cats_and_dogs/test'

image_path = []
for name in os.listdir(test_dir):
    test_image_path  = test_dir + '/' + name                                    # 각 이미지에 경로 설정
    image_path.append(test_image_path)
print(image_path)

HEIGHT = 224
WIDTH = 224

for path in image_path:                                                         # 테스트 이미지로 모델 성능 확인
    img = Image.open(path)                                                      # 이미지 파일 열기
    img = img.resize((HEIGHT, WIDTH))                                           # 이미지 사이즈 설정
    X = np.array(img)                                                           # 배열로 변환
    X = X/ 255                                                                  # 정규화
    X = X.reshape(1, HEIGHT, WIDTH, 3)                                          # 이미지 4차원화

    categories = ['cats', 'dogs']                                               # 레이블 설정
    pred = finetuned_model.predict(X)                                           # 모델 적용
    result = [np.argmax(value) for value in pred]                               # 최댓값 인덱스 반환

    print(categories[result[0]])                                                # 레이블이 추출
    print(np.max(pred))                                                         # 정확도 추출
    plt.imshow(img)                                                             # 이미지 시각화
    plt.show()